In [3]:
import os
import cv2
import hashlib
import shutil
from pathlib import Path
from tqdm import tqdm

# 🔹 Path to your original dataset
root = Path(r"C:\Users\paul\Downloads\archive (2)\MergedDataset")   # <-- change this to your dataset path

# 🔹 Path where invalid images will be moved
invalid_dir = Path(r"C:\PlantDataset_Invalid")
invalid_dir.mkdir(exist_ok=True)

print(f"Source dataset: {root}")
print(f"Invalid images will be moved to: {invalid_dir}")


Source dataset: C:\Users\paul\Downloads\archive (2)\MergedDataset
Invalid images will be moved to: C:\PlantDataset_Invalid


In [4]:
def is_blurry(image_path, threshold=20.0):
    """Detect blurry images using Laplacian variance."""
    try:
        img = cv2.imread(str(image_path))
        if img is None:
            return True  # unreadable file
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        variance = cv2.Laplacian(gray, cv2.CV_64F).var()
        return variance < threshold
    except Exception:
        return True

def file_hash(image_path):
    """Generate MD5 hash for image to find duplicates."""
    try:
        with open(image_path, "rb") as f:
            return hashlib.md5(f.read()).hexdigest()
    except Exception:
        return None


In [5]:
# Dictionaries to store hashes and issues
seen_hashes = {}
duplicate_images = []
blurry_images = []
corrupt_images = []

image_files = list(root.rglob("*.*"))
print(f"🔍 Scanning {len(image_files)} images...")

for img_path in tqdm(image_files):
    # Skip non-image files
    if img_path.suffix.lower() not in [".jpg", ".jpeg", ".png"]:
        continue

    # Check if file can be opened
    try:
        img = cv2.imread(str(img_path))
        if img is None:
            corrupt_images.append(img_path)
            continue
    except Exception:
        corrupt_images.append(img_path)
        continue

    # Check for duplicates
    h = file_hash(img_path)
    if h:
        if h in seen_hashes:
            duplicate_images.append(img_path)
        else:
            seen_hashes[h] = img_path

    # Check for blurriness
    if is_blurry(img_path, threshold=20.0):
        blurry_images.append(img_path)


🔍 Scanning 68715 images...


100%|████████████████████████████████████████████████████████████████████████████| 68715/68715 [55:19<00:00, 20.70it/s]


In [6]:
def move_images(image_list, reason):
    if not image_list:
        print(f"✅ No {reason} images found.")
        return
    subfolder = invalid_dir / reason
    subfolder.mkdir(exist_ok=True)
    for img_path in tqdm(image_list, desc=f"Moving {reason} images"):
        try:
            dst = subfolder / img_path.name
            shutil.move(str(img_path), str(dst))
        except Exception as e:
            print(f"⚠️ Error moving {img_path}: {e}")

move_images(corrupt_images, "corrupt")
move_images(duplicate_images, "duplicates")
move_images(blurry_images, "blurry")


✅ No corrupt images found.


Moving blurry images:  24%|█████████████▉                                            | 90/374 [00:00<00:00, 408.78it/s]

⚠️ Error moving C:\Users\paul\Downloads\archive (2)\MergedDataset\Wheat\Septoria blotch\PlantWild_wheat_septoria_blotch_blotch (43).jpg: [WinError 2] The system cannot find the file specified
⚠️ Error moving C:\Users\paul\Downloads\archive (2)\MergedDataset\Wheat\Septoria blotch\PlantWild_wheat_septoria_blotch_google_septoria (16).jpg: [WinError 2] The system cannot find the file specified
⚠️ Error moving C:\Users\paul\Downloads\archive (2)\MergedDataset\Wheat\Stripe rust\PlantWild_wheat_stripe_rust_Bing_0151.jpg: [WinError 2] The system cannot find the file specified
⚠️ Error moving C:\Users\paul\Downloads\archive (2)\MergedDataset\Wheat\Stripe rust\PlantWild_wheat_stripe_rust_Bing_0272.jpg: [WinError 2] The system cannot find the file specified


Moving blurry images:  55%|███████████████████████████████▌                         | 207/374 [00:00<00:00, 367.49it/s]

⚠️ Error moving C:\Users\paul\Downloads\archive (2)\MergedDataset\Soybean\Frog eye spot\PlantWild_soybean_frog_eye_leaf_spot_Bing_0217.jpg: [WinError 2] The system cannot find the file specified
⚠️ Error moving C:\Users\paul\Downloads\archive (2)\MergedDataset\Raspberry\Gray mold\PlantWild_raspberry_gray_mold_Google_0011.jpg: [WinError 2] The system cannot find the file specified
⚠️ Error moving C:\Users\paul\Downloads\archive (2)\MergedDataset\Pepper_bell\Blossom end rot\PlantWild_bell_pepper_blossom_end_rot_Google_0294.jpg: [WinError 2] The system cannot find the file specified
⚠️ Error moving C:\Users\paul\Downloads\archive (2)\MergedDataset\Pepper_bell\Frogeye spot\PlantWild_bell_pepper_frogeye_leaf_spot_Bing_0007.jpg: [WinError 2] The system cannot find the file specified
⚠️ Error moving C:\Users\paul\Downloads\archive (2)\MergedDataset\Pepper_bell\Frogeye spot\PlantWild_bell_pepper_frogeye_leaf_spot_Bing_0010.jpg: [WinError 2] The system cannot find the file specified
⚠️ Error mo

Moving blurry images:  87%|█████████████████████████████████████████████████▋       | 326/374 [00:00<00:00, 361.40it/s]

⚠️ Error moving C:\Users\paul\Downloads\archive (2)\MergedDataset\Corn\Rust\PlantWild_corn_rust_google_0096.jpg: [WinError 2] The system cannot find the file specified
⚠️ Error moving C:\Users\paul\Downloads\archive (2)\MergedDataset\Cauliflower\Bacterial soft rot\PlantWild_cauliflower_bacterial_soft_rot_Google_0016.jpg: [WinError 2] The system cannot find the file specified
⚠️ Error moving C:\Users\paul\Downloads\archive (2)\MergedDataset\Cabbage\Black rot\PlantWild_cabbage_black_rot_Google_0247.jpg: [WinError 2] The system cannot find the file specified
⚠️ Error moving C:\Users\paul\Downloads\archive (2)\MergedDataset\Blueberry\Anthracnose\PlantWild_blueberry_anthracnose_Google_0003.jpg: [WinError 2] The system cannot find the file specified


Moving blurry images: 100%|█████████████████████████████████████████████████████████| 374/374 [00:01<00:00, 360.79it/s]


In [7]:
print("\n🌿 Cleaning Summary")
print("--------------------------------------")
print(f"Total images scanned : {len(image_files)}")
print(f"Corrupt images moved : {len(corrupt_images)}")
print(f"Duplicate images moved: {len(duplicate_images)}")
print(f"Blurry images moved  : {len(blurry_images)}")
print("--------------------------------------")
print(f"🗂️ Invalid images safely stored in: {invalid_dir}")




🌿 Cleaning Summary
--------------------------------------
Total images scanned : 68715
Corrupt images moved : 0
Duplicate images moved: 493
Blurry images moved  : 374
--------------------------------------
🗂️ Invalid images safely stored in: C:\PlantDataset_Invalid
